In [87]:
import zipfile
import io
import pandas as pd
from IPython.display import display
from collections import defaultdict

zipped_data_path = "../data/raw_data/ucsd-cse-151b-class-competition.zip"

dataframes = defaultdict(pd.DataFrame)
with zipfile.ZipFile(zipped_data_path, "r") as zip:
    for filename in zip.namelist():
        if filename.endswith(".csv"):
            with zip.open(filename) as f:
                dataframes.update({ filename : pd.read_csv(io.TextIOWrapper(f)) })

                # Lets take a look at the files
                print(f"FILE: {filename}")
                display(dataframes[filename].info())
                display(dataframes[filename].head())

FILE: metaData_taxistandsID_name_GPSlocation.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         63 non-null     int64  
 1   Descricao  63 non-null     object 
 2   Latitude   63 non-null     object 
 3   Longitude  62 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.1+ KB


None

,ID,Descricao,Latitude,Longitude
0,1,Agra,41.1771457135,-8.609670
1,2,Alameda,41.15618964,-8.591064
2,3,Aldoar,41.1705249231,-8.665876
3,4,AlfÃ¢ndega,41.1437639911,-8.621803
4,5,Amial,41.1835097223,-8.612726


FILE: sampleSubmission.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   TRIP_ID      320 non-null    object
 1   TRAVEL_TIME  320 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.1+ KB


None

...

None

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,T1,B,NaN,15.0,20000542,1408039037,A,False
1,T2,B,NaN,57.0,20000108,1408038611,A,False
2,T3,B,NaN,15.0,20000370,1408038568,A,False
3,T4,B,NaN,53.0,20000492,1408039090,A,False
4,T5,B,NaN,18.0,20000621,1408039177,A,False


FILE: train.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710670 entries, 0 to 1710669
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   TRIP_ID       int64  
 1   CALL_TYPE     object 
 2   ORIGIN_CALL   float64
 3   ORIGIN_STAND  float64
 4   TAXI_ID       int64  
 5   TIMESTAMP     int64  
 6   DAY_TYPE      object 
 7   MISSING_DATA  bool   
 8   POLYLINE      object 
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 106.0+ MB


None

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [88]:
TAXI_METADATA = dataframes["metaData_taxistandsID_name_GPSlocation.csv"]
SAMPLE_SUBMISSION_DF = dataframes["sample_submission.csv"]
TEST_PUBLIC_DF = dataframes["test_public.csv"]
TRAIN_DF = dataframes["train.csv"]

## **train.csv**

| Column Name | Description |
| --- | --- |
| TRIP_ID | (String) - An unique identifier for each trip |
| CALL_TYPE | (char) - Category of the ride. It may contain one of three possible values: 'A' if this trip was dispatched from the central, 'B' if this trip was demanded directly to a taxi driver on a specific stand, 'C' otherwise (i.e. a trip demanded on a random street) |
| ORIGIN_CALL | (integer) - An unique identifier for the phone number to call the taxi. It identifies the trip's customer if CALL_TYPE='A'. Otherwise, it assumes a NULL value |
| ORIGIN_STAND | (integer) - An unique identifier for the taxi stand. It identifies the starting point of the trip if CALL_TYPE='B'. Otherwise, it assumes a NULL value |
| TAXI_ID | (integer) - An unique identifier for the taxi that performed each trip |
| TIMESTAMP | (integer) - Unix Timestamp (in seconds). It identifies the trip's start |
| DAY_TYPE | (char) - It identifies the daytype of the trip's start. It assumes one of three possible values: 'A' if this trip started on a normal day or weekend, 'B' if this trip started on a holiday or other special day, 'C' if the trip started on a day before a type-B day |
| MISSING_DATA | (Boolean) - It is FALSE when the GPS data stream is complete and TRUE whenever one (or more) locations are missing |
| POLYLINE | (String) - A list of GPS coordinates (i.e. WGS84 format) mapped as a string. The beginning and the end of the string are identified with brackets (i.e. [ and ]). Each pair of coordinates is also identified by the same brackets as [LONGITUDE, LATITUDE]. The coordinates were recorded every 15 seconds during the trip. The first item represents the starting point and the last item corresponds to the destination |

In [89]:
TRAIN_DF.describe()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP
count,1.710670e+06,364770.000000,806579.000000,1.710670e+06,1.710670e+06
mean,1.388622e+18,24490.363018,30.272381,2.000035e+07,1.388622e+09
std,9.180944e+15,19624.290043,17.747840,2.112405e+02,9.180944e+06
min,1.372637e+18,2001.000000,1.000000,2.000000e+07,1.372637e+09
25%,1.380731e+18,6593.000000,15.000000,2.000017e+07,1.380731e+09
50%,1.388493e+18,18755.000000,27.000000,2.000034e+07,1.388493e+09
75%,1.396750e+18,40808.000000,49.000000,2.000052e+07,1.396750e+09
max,1.404173e+18,63884.000000,63.000000,2.000098e+07,1.404173e+09


In [90]:
TRAIN_DF.isnull().sum()

TRIP_ID               0
CALL_TYPE             0
ORIGIN_CALL     1345900
ORIGIN_STAND     904091
TAXI_ID               0
TIMESTAMP             0
DAY_TYPE              0
MISSING_DATA          0
POLYLINE              0
dtype: int64

In [91]:
# How many GPS data streams are complete vs incomplete
TRAIN_DF["MISSING_DATA"].value_counts()

MISSING_DATA
False    1710660
True          10
Name: count, dtype: int64

In [92]:
# Lets remove the missing data
TRAIN_DF = TRAIN_DF[TRAIN_DF["MISSING_DATA"] != True]
TRAIN_DF["MISSING_DATA"].value_counts()

MISSING_DATA
False    1710660
Name: count, dtype: int64

In [93]:
# Percentage of each call type
TRAIN_DF["CALL_TYPE"].value_counts(normalize=True)

CALL_TYPE
B    0.478107
C    0.308660
A    0.213233
Name: proportion, dtype: float64

In [94]:
# Percentage of each day type
TRAIN_DF["DAY_TYPE"].value_counts(normalize=True)

DAY_TYPE
A    1.0
Name: proportion, dtype: float64

In [95]:
import json

def calculate_travel_time(polyline: str) -> int:
    """
    Calculates the travel time of a trip. Is defined as
    (number of points - 1) * 15 seconds.
    
    :param polyline: The polyline of the trip.
    :return: The travel time of the trip.
    """
    return (len(json.loads(polyline)) - 1) * 15

TRAIN_DF["TRAVEL_TIME"] = TRAIN_DF["POLYLINE"].apply(calculate_travel_time)
TRAIN_DF.head()

C:\Users\Tyler Holston\AppData\Local\Temp\ipykernel_36848\466646351.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRAIN_DF["TRAVEL_TIME"] = TRAIN_DF["POLYLINE"].apply(calculate_travel_time)


,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,TRAVEL_TIME
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",330
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",270
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",960
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",630
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",420


In [96]:
# Lets split DATA_TYPE and CALL_TYPE into three new binary features 
# i.e. if DATA_TYPE = 'A' -> DATA_TYPE_A = 1, DATA_TYPE_B = 0, DATA_TYPE_C = 0
#day_type_dummies = pd.get_dummies(TRAIN_DF['DAY_TYPE'], prefix='DAY_TYPE')
#missing_cols = set(['DAY_TYPE_A', 'DAY_TYPE_B', 'DAY_TYPE_C']) - set(day_type_dummies.columns)
#for col in missing_cols:
#    day_type_dummies[col] = False
#TRAIN_DF = pd.concat([TRAIN_DF, day_type_dummies], axis=1)

call_type_dummies = pd.get_dummies(TRAIN_DF['CALL_TYPE'], prefix='CALL_TYPE')
missing_cols = set(['CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C']) - set(call_type_dummies.columns)
for col in missing_cols:
    call_type_dummies[col] = False
TRAIN_DF = pd.concat([TRAIN_DF, call_type_dummies], axis=1)

# delete the old columns now that we don't need them anymore
TRAIN_DF.drop('DAY_TYPE', axis=1, inplace=True)
TRAIN_DF.drop('CALL_TYPE', axis=1, inplace=True)

TRAIN_DF.head()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,MISSING_DATA,POLYLINE,TRAVEL_TIME,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C
0,1372636858620000589,NaN,NaN,20000589,1372636858,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",330,False,False,True
1,1372637303620000596,NaN,7.0,20000596,1372637303,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",270,False,True,False
2,1372636951620000320,NaN,NaN,20000320,1372636951,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",960,False,False,True
3,1372636854620000520,NaN,NaN,20000520,1372636854,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",630,False,False,True
4,1372637091620000337,NaN,NaN,20000337,1372637091,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",420,False,False,True


In [97]:
# Lets try to get some additional features from the timestamp column
TRAIN_DF['TIMESTAMP'] = pd.to_datetime(TRAIN_DF['TIMESTAMP'], unit='s')

# Extract year, month, day of the week, hour, and minute
TRAIN_DF['YEAR'] = TRAIN_DF['TIMESTAMP'].dt.year
TRAIN_DF['MONTH'] = TRAIN_DF['TIMESTAMP'].dt.month
TRAIN_DF['DAY_OF_WEEK'] = TRAIN_DF['TIMESTAMP'].dt.dayofweek
TRAIN_DF['HOUR'] = TRAIN_DF['TIMESTAMP'].dt.hour

# One-hot encode year, month, day of the week, and hour
TRAIN_DF = pd.concat([TRAIN_DF, pd.get_dummies(TRAIN_DF['YEAR'], prefix='YEAR')], axis=1)
TRAIN_DF = pd.concat([TRAIN_DF, pd.get_dummies(TRAIN_DF['MONTH'], prefix='MONTH')], axis=1)
TRAIN_DF = pd.concat([TRAIN_DF, pd.get_dummies(TRAIN_DF['DAY_OF_WEEK'], prefix='DAY_OF_WEEK')], axis=1)
TRAIN_DF = pd.concat([TRAIN_DF, pd.get_dummies(TRAIN_DF['HOUR'], prefix='HOUR')], axis=1)


# Drop the original TIMESTAMP and DAY_OF_WEEK columns
TRAIN_DF.drop(['TIMESTAMP', 'DAY_OF_WEEK', 'YEAR', 'MONTH', 'HOUR'], axis=1, inplace=True)

# We could do referene encoding to speed up train time in the future 
# For day of the week, Sunday is 0,0,0,0,0,0 - Monday is 1,0,0,0,0,0 - Tuesday is 0,1,0,0,0,0 - etc.
# TRAIN_DF.drop(['DAY_OF_WEEK_0', 'MONTH_1', 'HOUR_0', 'YEAR_2013'], axis=1, inplace=True)

pd.set_option('display.max_columns', 100)
print(TRAIN_DF.columns)

Index(['TRIP_ID', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'MISSING_DATA',
       'POLYLINE', 'TRAVEL_TIME', 'CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C',
       'YEAR_2013', 'YEAR_2014', 'MONTH_1', 'MONTH_2', 'MONTH_3', 'MONTH_4',
       'MONTH_5', 'MONTH_6', 'MONTH_7', 'MONTH_8', 'MONTH_9', 'MONTH_10',
       'MONTH_11', 'MONTH_12', 'DAY_OF_WEEK_0', 'DAY_OF_WEEK_1',
       'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5',
       'DAY_OF_WEEK_6', 'HOUR_0', 'HOUR_1', 'HOUR_2', 'HOUR_3', 'HOUR_4',
       'HOUR_5', 'HOUR_6', 'HOUR_7', 'HOUR_8', 'HOUR_9', 'HOUR_10', 'HOUR_11',
       'HOUR_12', 'HOUR_13', 'HOUR_14', 'HOUR_15', 'HOUR_16', 'HOUR_17',
       'HOUR_18', 'HOUR_19', 'HOUR_20', 'HOUR_21', 'HOUR_22', 'HOUR_23'],
      dtype='object')


In [98]:
# TEST_PUBLIC_CSV feature extraction
call_type_dummies = pd.get_dummies(TEST_PUBLIC_DF['CALL_TYPE'], prefix='CALL_TYPE')
missing_cols = set(['CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C']) - set(call_type_dummies.columns)
for col in missing_cols:
    call_type_dummies[col] = False
TEST_PUBLIC_DF = pd.concat([TEST_PUBLIC_DF, call_type_dummies], axis=1)

# delete the old columns now that we don't need them anymore
TEST_PUBLIC_DF.drop('DAY_TYPE', axis=1, inplace=True)
TEST_PUBLIC_DF.drop('CALL_TYPE', axis=1, inplace=True)

TEST_PUBLIC_DF.head()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,MISSING_DATA,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C
0,T1,NaN,15.0,20000542,1408039037,False,False,True,False
1,T2,NaN,57.0,20000108,1408038611,False,False,True,False
2,T3,NaN,15.0,20000370,1408038568,False,False,True,False
3,T4,NaN,53.0,20000492,1408039090,False,False,True,False
4,T5,NaN,18.0,20000621,1408039177,False,False,True,False


In [99]:
# Lets try to get some additional features from the timestamp column
hour_columns = ['HOUR_' + str(i) for i in range(24)]
day_columns = ['DAY_OF_WEEK_' + str(i) for i in range(7)]
month_columns = ['MONTH_' + str(i) for i in range(1,13)]
year_columns = ['YEAR_2013', 'YEAR_2014']

TEST_PUBLIC_DF['TIMESTAMP'] = pd.to_datetime(TEST_PUBLIC_DF['TIMESTAMP'], unit='s')

# Extract year, month, day of the week, hour, and minute
TEST_PUBLIC_DF['YEAR'] = TEST_PUBLIC_DF['TIMESTAMP'].dt.year
TEST_PUBLIC_DF['MONTH'] = TEST_PUBLIC_DF['TIMESTAMP'].dt.month
TEST_PUBLIC_DF['DAY_OF_WEEK'] = TEST_PUBLIC_DF['TIMESTAMP'].dt.dayofweek
TEST_PUBLIC_DF['HOUR'] = TEST_PUBLIC_DF['TIMESTAMP'].dt.hour

# One-hot encode year, month, day of the week, and hour
TEST_PUBLIC_DF = pd.concat([TEST_PUBLIC_DF, pd.get_dummies(TEST_PUBLIC_DF['YEAR'], prefix='YEAR')], axis=1)
TEST_PUBLIC_DF = pd.concat([TEST_PUBLIC_DF, pd.get_dummies(TEST_PUBLIC_DF['MONTH'], prefix='MONTH')], axis=1)
TEST_PUBLIC_DF = pd.concat([TEST_PUBLIC_DF, pd.get_dummies(TEST_PUBLIC_DF['DAY_OF_WEEK'], prefix='DAY_OF_WEEK')], axis=1)
TEST_PUBLIC_DF = pd.concat([TEST_PUBLIC_DF, pd.get_dummies(TEST_PUBLIC_DF['HOUR'], prefix='HOUR')], axis=1)

# Fill missing dummy columns with False
for column in hour_columns:
    if column not in TEST_PUBLIC_DF.columns:
        TEST_PUBLIC_DF[column] = False

for column in day_columns:
    if column not in TEST_PUBLIC_DF.columns:
        TEST_PUBLIC_DF[column] = False

for column in month_columns:
    if column not in TEST_PUBLIC_DF.columns:
        TEST_PUBLIC_DF[column] = False

for column in year_columns:
    if column not in TEST_PUBLIC_DF.columns:
        TEST_PUBLIC_DF[column] = False

# Drop the original TIMESTAMP and DAY_OF_WEEK columns
TEST_PUBLIC_DF.drop(['TIMESTAMP', 'DAY_OF_WEEK', 'YEAR', 'MONTH', 'HOUR'], axis=1, inplace=True)
TEST_PUBLIC_DF = TEST_PUBLIC_DF.reindex(columns=TRAIN_DF.columns)
TEST_PUBLIC_DF = TEST_PUBLIC_DF.drop("TRAVEL_TIME", axis=1)

In [100]:
modified_train_zip = '../data/clean_data/class-competition.zip'

with zipfile.ZipFile(modified_train_zip, 'w') as zip:
    zip.writestr("train.csv", TRAIN_DF.to_csv(index=False))
    zip.writestr("test_public.csv", TEST_PUBLIC_DF.to_csv(index=False))